### 함수, 모듈 import

In [154]:
import os
import sys
import urllib.request
import json 
import datetime 
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.request import urlopen 
from glob import glob
import operator
import seaborn as sns
from xml.etree.ElementTree import parse
import time
from bs4 import BeautifulSoup
from bs4.element import NavigableString

#!pip install xlsxwriter

client_id = "u1IEUf8fHCPaCpVyZGgt"
client_secret = "dL13QH3nc5"

import platform
import matplotlib.pyplot as plt 
from matplotlib import font_manager, rc

rc("font", family="Arial Unicode MS")
plt.rcParams["axes.unicode_minus"] = False 

In [40]:
def gen_search_url(api_node, search_text, start_num, disp_num):
    base = "https://openapi.naver.com/v1/search"
    node = "/" + api_node + ".json"
    param_titl = "?d_titl=" + urllib.parse.quote(search_text)
    param_start = "&start=" + str(start_num)
    param_disp = "&display=" + str(disp_num)
    param_catg = "&d_catg=280"

    return base + node + param_titl + param_start + param_disp + param_catg

In [41]:
url = gen_search_url('book_adv', 'c++', 1, 100)
url

'https://openapi.naver.com/v1/search/book_adv.json?d_titl=c%2B%2B&start=1&display=100&d_catg=280'

In [61]:
# get_result_onpage

def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    print("[%s] Url Request Success" % datetime.datetime.now())
    time.sleep(1)
    return json.loads(response.read().decode("utf-8"))


In [43]:
# delete_tag

def delete_tag(input_str):
    input_str = input_str.replace("<b>", "")
    input_str = input_str.replace("</b>", "")
    return input_str 

In [44]:
# get_fields

def get_fields(json_data):
    isbn = [each["isbn"] for each in json_data["items"]]
    title = [delete_tag(each["title"]) for each in json_data["items"]]
    author = [delete_tag(each["author"]) for each in json_data["items"]]
    price = [each["price"] for each in json_data["items"]]
    publisher = [delete_tag(each["publisher"]) for each in json_data["items"]]
    pubdate = [each["pubdate"] for each in json_data["items"]]
    link = [each["link"] for each in json_data["items"]]
    
    result_pd = pd.DataFrame({
        "isbn": isbn, 
        "title": title, 
        "author": author, 
        "price": price,
        "publisher": publisher,
        "pubdate": pubdate,
        "link": link
    }, columns=["isbn", "title", "author", "price", "publisher", "pubdate", "link"])
    return result_pd

In [48]:
def actMain(search_text):

    result_mol = []
    
    for n in range(1, 1000, 100):
        url = gen_search_url('book_adv', search_text, n, 100)
        json_result = get_result_onpage(url)
        pd_result = get_fields(json_result)

        result_mol.append(pd_result)
        
        
    return pd.concat(result_mol)
    # result_mol.reset_index(drop=True, inplace=True)

In [52]:
def toExcel(file_name):
    writer = pd.ExcelWriter('data_211021/search_'+file_name, engine="xlsxwriter")
    result_mol.to_excel(writer, sheet_name="Sheet1")

    workbook = writer.book 
    worksheet = writer.sheets["Sheet1"]
    worksheet.set_column("A:A", 4)
    worksheet.set_column("B:B", 25)
    worksheet.set_column("C:C", 60)
    worksheet.set_column("D:D", 10)
    worksheet.set_column("E:E", 10)
    worksheet.set_column("F:F", 20)
    worksheet.set_column("G:G", 15)
    worksheet.set_column("H:H", 60)

    worksheet.conditional_format("E2:E1001", {"type": "3_color_scale"})
    writer.save()

### * 20가지 언어에 대한 검색 결과 저장하기 실행

In [54]:
searchWord1 = ['python', '파이썬', 'C언어', 'java', 'C++', 'C#', 'Visual Basic', 'SQL', 'PHP', 'R프로그래밍', 'R통계']
searchWord2 = ['assambly', '어셈블리', 'classic visual basic', 'golang', 'matlab', 'groovy', 'ruby', 'swift', 'fortran', 'perl', '델파이', '오브젝트 파스칼', 'delphi', 'object pascal']

for word in searchWord1:
    result_mol=actMain(word)
    toExcel(word+'.xlsx')
    print(word)


[2021-10-21 17:09:11.873909] Url Request Success
[2021-10-21 17:09:11.965405] Url Request Success
[2021-10-21 17:09:12.031724] Url Request Success
[2021-10-21 17:09:12.092403] Url Request Success
[2021-10-21 17:09:12.152635] Url Request Success
[2021-10-21 17:09:12.223928] Url Request Success
[2021-10-21 17:09:12.304184] Url Request Success
[2021-10-21 17:09:12.389221] Url Request Success
[2021-10-21 17:09:12.449478] Url Request Success
[2021-10-21 17:09:12.521803] Url Request Success
python
[2021-10-21 17:09:13.006986] Url Request Success
[2021-10-21 17:09:13.417581] Url Request Success
[2021-10-21 17:09:13.767362] Url Request Success
[2021-10-21 17:09:14.229489] Url Request Success
[2021-10-21 17:09:14.641760] Url Request Success
[2021-10-21 17:09:15.054092] Url Request Success
[2021-10-21 17:09:15.350291] Url Request Success
[2021-10-21 17:09:15.425476] Url Request Success
[2021-10-21 17:09:15.507846] Url Request Success
[2021-10-21 17:09:15.573998] Url Request Success
파이썬
[2021-10-

# 문제2) 중복되는 데이터 제거

### 1) 데이터 다시 불러와서 한 데이터 프레임에 담기

In [63]:
files=glob("data_211021/search_*.xlsx")
files

['data_211021/search_PHP.xlsx',
 'data_211021/search_matlab.xlsx',
 'data_211021/search_fortran.xlsx',
 'data_211021/search_SQL.xlsx',
 'data_211021/search_python.xlsx',
 'data_211021/search_java.xlsx',
 'data_211021/search_groovy.xlsx',
 'data_211021/search_delphi.xlsx',
 'data_211021/search_swift.xlsx',
 'data_211021/search_R통계.xlsx',
 'data_211021/search_ruby.xlsx',
 'data_211021/search_classic visual basic.xlsx',
 'data_211021/search_assambly.xlsx',
 'data_211021/search_object pascal.xlsx',
 'data_211021/search_C#.xlsx',
 'data_211021/search_Visual Basic.xlsx',
 'data_211021/search_오브젝트 파스칼.xlsx',
 'data_211021/search_R프로그래밍.xlsx',
 'data_211021/search_어셈블리.xlsx',
 'data_211021/search_파이썬.xlsx',
 'data_211021/search_C++.xlsx',
 'data_211021/search_golang.xlsx',
 'data_211021/search_C언어.xlsx',
 'data_211021/search_perl.xlsx',
 'data_211021/search_델파이.xlsx']

In [64]:
tmp_raw = []

for file_name in files:
    tmp = pd.read_excel(file_name, index_col=0)
    tmp_raw.append(tmp)

search_raw = pd.concat(tmp_raw)

In [65]:
search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5673 entries, 0 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   isbn       5671 non-null   object 
 1   title      5673 non-null   object 
 2   author     5665 non-null   object 
 3   price      5671 non-null   float64
 4   publisher  5673 non-null   object 
 5   pubdate    5673 non-null   object 
 6   link       5673 non-null   object 
dtypes: float64(1), object(6)
memory usage: 354.6+ KB


### 2) 중복되는 행을 제거하고 고유한 값만 남기기

In [66]:
search_raw.duplicated(['isbn'])

0     False
1     False
2     False
3     False
4     False
      ...  
96    False
97    False
98    False
99    False
0     False
Length: 5673, dtype: bool

In [67]:
search = search_raw.drop_duplicates(['isbn'], keep='first')

In [68]:
search.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5403 entries, 0 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   isbn       5402 non-null   object 
 1   title      5403 non-null   object 
 2   author     5397 non-null   object 
 3   price      5401 non-null   float64
 4   publisher  5403 non-null   object 
 5   pubdate    5403 non-null   object 
 6   link       5403 non-null   object 
dtypes: float64(1), object(6)
memory usage: 337.7+ KB


In [69]:
search.tail()

,isbn,title,author,price,publisher,pubdate,link
96,898577056X 9788985770569,따라해보세요 델파이 2.0,김중완,9000.0,한글과컴퓨터,19960618,http://book.naver.com/bookdb/book_detail.php?b...
97,8974672308 9788974672300,마스터링 델파이 2 상,Marco Cantu,26000.0,삼각형,19960820,http://book.naver.com/bookdb/book_detail.php?b...
98,8974671476 9788974671471,이것이 델파이다 상,Marco Cantu,25000.0,삼각형,19960101,http://book.naver.com/bookdb/book_detail.php?b...
99,8940668286 9788940668283,델파이 2.0,조남균,15000.0,크라운출판사,19970410,http://book.naver.com/bookdb/book_detail.php?b...
0,8975880281 9788975880285,델파이 이렇게 사용한다,한민영,13000.0,높이깊이,19950901,http://book.naver.com/bookdb/book_detail.php?b...


# 문제 3) 언어별 출판물 양으로 순위 매기기

In [71]:
searchWord = searchWord1 + searchWord2
searchWord

['python',
 '파이썬',
 'C언어',
 'java',
 'C++',
 'C#',
 'Visual Basic',
 'SQL',
 'PHP',
 'R프로그래밍',
 'R통계',
 'assambly',
 '어셈블리',
 'classic visual basic',
 'golang',
 'matlab',
 'groovy',
 'ruby',
 'swift',
 'fortran',
 'perl',
 '델파이',
 '오브젝트 파스칼',
 'delphi',
 'object pascal']

In [72]:
searchWord[4] = 'C\+\+'
searchWord[5] = 'C\#'

In [73]:
# searchWord 리스트 활용해서 cnt 열을 포함한 데이터 프레임 만들기

df = pd.DataFrame(columns=['idx','cnt'])

for i in searchWord : 
    idx=i
    cnt=sum(search['title'].str.contains(i, case=False))
    df = df.append(pd.DataFrame([[idx,cnt]],columns=['idx','cnt']),ignore_index=True)

df

,idx,cnt
0,python,204
1,파이썬,668
2,C언어,510
3,java,739
4,C\+\+,979
5,C\#,300
6,Visual Basic,339
7,SQL,453
8,PHP,197
9,R프로그래밍,4


In [121]:
# 검색시 중복되었던 항목 제거

df_1=df.copy()
df_1.iloc[0].cnt=(df.iloc[0]+df.iloc[1]).cnt #파이썬 합치기
df_1.iloc[1].cnt=0
df_1.iloc[9].cnt=(df.iloc[9]+df.iloc[10]).cnt #R 합치기
df_1.iloc[10].cnt=0
df_1.iloc[11].cnt=(df.iloc[11]+df.iloc[12]).cnt #어셈블리 합치기
df_1.iloc[12].cnt=0
df_1.iloc[21].cnt=(df.iloc[21]+df.iloc[22]+df.iloc[23]+df.iloc[24]).cnt #델파이, 오브젝트 파스칼 합치기
df_1.iloc[22].cnt=0
df_1.iloc[23].cnt=0
df_1.iloc[24].cnt=0


In [122]:
df_1.drop([1,10,12,13,16,22,23,24],inplace=True)
df_1.reset_index(drop=True)

,idx,cnt
0,python,872
1,C언어,510
2,java,739
3,C\+\+,979
4,C\#,300
5,Visual Basic,339
6,SQL,453
7,PHP,197
8,R프로그래밍,7
9,assambly,140


In [125]:
df_1.iloc[16].idx = 'Delphi/Object Pascal'

In [126]:
# rank
df_2 = df_1.copy()
df_2['rank'] = df_2['cnt'].rank(ascending=False)

df_2.sort_values(by='rank', inplace=True)
df_2

,idx,cnt,rank
4,C\+\+,979,1.0
0,python,872,2.0
3,java,739,3.0
2,C언어,510,4.0
7,SQL,453,5.0
6,Visual Basic,339,6.0
5,C\#,300,7.0
19,fortran,234,8.0
8,PHP,197,9.0
15,matlab,184,10.0


In [205]:
url=search.iloc[8].link
response = urlopen(url)
soup = BeautifulSoup(response, "html.parser")
#print(soup.prettify())

target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
print(target_tag.text)




페이지 420|ISBN  9788970503684


In [ ]:
len(search)

5403

In [220]:
page_df = pd.DataFrame(index=range(5403),columns=['page'])

for i in range(1000):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [224]:
for i in range(1000,2000):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [229]:
for i in range(2000,3000):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [230]:
for i in range(3000,4000):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [232]:
for i in range(4000,5000):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [233]:
for i in range(5000,5403):

    url=search.iloc[i].link
    response = urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    try :
        target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(3)")
        page = int(target_tag.text.rstrip()[4:8])
    except :
        try :
            target_tag = soup.select_one("#container > div.spot > div.book_info > div.book_info_inner > div:nth-child(4)")
            page = int(target_tag.text.rstrip()[4:8])
        except : 
            page = ''

    page_df.iloc[i].page = page

In [235]:
page_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5403 entries, 0 to 5402
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   page    5403 non-null   object
dtypes: object(1)
memory usage: 42.3+ KB


In [240]:
search = search.reset_index(drop=True)

In [241]:
search_page = pd.concat([search, page_df], axis=1)

In [244]:
result_mol=search_page
toExcel('total.xlsx')

In [248]:
search = pd.read_excel('data_211021/search_total.xlsx', index_col=0)

In [249]:
search.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5403 entries, 0 to 5402
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   isbn       5402 non-null   object 
 1   title      5403 non-null   object 
 2   author     5397 non-null   object 
 3   price      5401 non-null   float64
 4   publisher  5403 non-null   object 
 5   pubdate    5403 non-null   object 
 6   link       5403 non-null   object 
 7   page       5288 non-null   float64
dtypes: float64(2), object(6)
memory usage: 379.9+ KB
